# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  classification_report
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///db_udacity_ETL.db')
df = pd.read_sql_table('TF_Messages',con=engine)
#creation of our X variable containing only messages
X = df['message']
#creation of our X variable with all the columns except the messages
Y =df.drop(columns=['message'])

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    '''
    Returns the list of cleaned words from the input message

            Parameters:
                    text (string): the message (text) to clean

            Returns:
                    clean_tokens (array): array of the cleaned words from the input message.
    '''
    tokens = word_tokenize(text) #We split the message into word
    lemmatizer = WordNetLemmatizer() #Instantiation of WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # Lemmatize, lower_cap and remove spaces
        clean_tokens.append(clean_tok) #append clean words to the results clean_tokens

    return clean_tokens

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/var/folders/7v/57lr0j397qb3kwvl259j828h0000gn/T/ipykernel_22580/3010515617.py:1: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#We create our Pipeline 
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Create of the train/test dataset from X and Y variables
X_train, X_test, y_train, y_test = train_test_split(X, Y)
#we fit our model using the pipeline and the train variables.
pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
#we predict and display classification report
y_pred = pipeline.predict(X_test)
for index,c in enumerate(y_test.columns):
    print ("---" +c+ "---")
    print(classification_report (y_test.iloc[:,index].values, y_pred[:,index]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#to know with paramaters can be tweaked
pipeline.get_params().keys()

In [ ]:
#definition of the parameters with the values to be tested
parameters = {
    
    'clf__estimator__n_estimators': [100,150,200],
    'clf__estimator__max_depth' : range (3,10,15)
}
#creation of the gridSearch with the usage of all cpu available
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1,verbose=2) 
#we fit the GridSearch to identfy the best parameters
cv.fit (X_train,y_train)

In [ ]:
#let's see what are the best parameters
print(cv.best_params_)
#store the best model 
cv_best = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_cv = cv_best.predict(X_test)
for index,c in enumerate(y_test.columns):
    print ("---" +c+ "---")
    print(classification_report (y_test.iloc[:,index].values, y_pred_cv[:,index]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [8]:

#try with a Gradient Boosting Classifier
pipeline_GB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('ada_clf', MultiOutputClassifier(AdaBoostClassifier()))])

In [9]:
#to know with paramaters can be tweaked
pipeline_GB.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'ada_clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'ada_clf__estimator__algorithm', 'ada_clf__estimator__base_estimator', 'ada_clf__estimator__estimator', 'ada_clf__estimator__learning_rate', 'ada_clf__estimator__n_estimators', 'ada_clf__estimator__random_state', 'ada_clf__estimator', 'ada_clf__n_jobs'])

In [10]:

parameters_GB = {  
    'ada_clf__estimator__n_estimators': [100,150,200],
    'ada_clf__estimator__learning_rate' : [0.5,1,1.5]
}
#creation of the gridSearch with the usage of all cpu available and Fit it
cv_GB= GridSearchCV(pipeline_GB, param_grid=parameters_GB, n_jobs=-1,verbose=2) 
cv_GB.fit (X_train,y_train)
#let's see what are the best parameters
print(cv_GB.best_params_)
#store the best model 
cv_GB_best = cv_GB.best_estimator_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-package

[CV] END ada_clf__estimator__learning_rate=0.5, ada_clf__estimator__n_estimators=100; total time= 3.0min


/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END ada_clf__estimator__learning_rate=0.5, ada_clf__estimator__n_estimators=100; total time= 3.0min


...

[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=150; total time= 4.5min


/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=150; total time= 4.5min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=150; total time= 4.5min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=150; total time= 4.3min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=200; total time= 5.4min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=200; total time= 5.4min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=200; total time= 5.3min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=200; total time= 5.0min
[CV] END ada_clf__estimator__learning_rate=1.5, ada_clf__estimator__n_estimators=200; total time= 4.9min


/Users/benjaminscaillierez/miniforge3/envs/udacityM1/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'ada_clf__estimator__learning_rate': 0.5, 'ada_clf__estimator__n_estimators': 200}


In [11]:
#Let's predict with our new model and see the resuls with classification report
y_pred = cv_GB_best.predict(X_test)
for index,c in enumerate(y_test.columns):
    print ("---" +c+ "---")
    print(classification_report (y_test.iloc[:,index].values, y_pred[:,index]))

---related---
              precision    recall  f1-score   support

           0       0.69      0.40      0.50      1536
           1       0.84      0.95      0.89      5018

    accuracy                           0.82      6554
   macro avg       0.76      0.67      0.70      6554
weighted avg       0.80      0.82      0.80      6554

---request---
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5440
           1       0.81      0.57      0.67      1114

    accuracy                           0.91      6554
   macro avg       0.87      0.77      0.81      6554
weighted avg       0.90      0.91      0.90      6554

---offer---
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99 

In [ ]:
cv_GB.best_params_

### 9. Export your model as a pickle file

In [12]:
#with save our model under the name bs_classifier.pkl
with open ("bs_classifier.pkl",'wb') as f:
    pickle.dump(cv_GB_best,f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.